In [82]:
import gradio as gr
import librosa
import numpy as np
import joblib
import cv2
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder

In [83]:
cnn_model = load_model("cnn_raw_model.h5")
cnn_featured_model = load_model("cnn_featured_model.h5")

In [84]:
rf_model = joblib.load("ml_rf_model.pkl")  
scaler = joblib.load("scaler.pkl")   
label_encoder = joblib.load("label_encoder.pkl")

In [85]:
label_encoder = LabelEncoder()
label_encoder.fit(["blues", "classical", "country", "disco", "hiphop", "jazz", "metal", "pop", "reggae", "rock"])

LabelEncoder()

In [86]:
def preprocess_audio(file_data, sr=22050, img_size=(128, 128)):
    try:
        y, sr = librosa.load(file_data, sr=sr, duration=30)

        mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

        mel_spec_normalized = mel_spec_db - np.min(mel_spec_db)
        mel_spec_normalized /= np.max(mel_spec_normalized)

        mel_spec_resized = cv2.resize(mel_spec_normalized, img_size)

        mel_spec_input = np.expand_dims(mel_spec_resized, axis=(0, -1))
        return mel_spec_input
    except Exception as e:
        print(f"Error processing audio: {e}")
        return None

In [87]:
def extract_audio_features(file_path, sr=22050):
    try:
        
        y, sr = librosa.load(file_path, sr=sr, duration=30)

        spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr).mean()
        spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr).mean()
        spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr, roll_percent=0.85).mean()

        zero_crossing_rate = librosa.feature.zero_crossing_rate(y).mean()
        rmse = librosa.feature.rms(y=y).mean()
        temporal_entropy = -np.sum((y**2) * np.log2(y**2 + 1e-10))

        features = {
            'Spectral Centroid': spectral_centroid,
            'Spectral Bandwidth': spectral_bandwidth,
            'Spectral Roll-Off': spectral_rolloff,
            'Zero Crossing Rate': zero_crossing_rate,
            'RMS Energy': rmse,
            'Temporal Entropy': temporal_entropy
        }
        return features
    except Exception as e:
        print(f"Error processing audio: {e}")
        return None

In [88]:
def preprocess_feature_engineered_spectrogram(file_path):
    y, sr = librosa.load(file_path, sr=22050, duration=30)
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

    mel_spec_db = (mel_spec_db - np.min(mel_spec_db)) / (np.max(mel_spec_db) - np.min(mel_spec_db))

    mel_spec_resized = cv2.resize(mel_spec_db, (128, 128))
    
    mel_spec_input = np.stack([mel_spec_resized] * 3, axis=-1)  
    mel_spec_input = np.expand_dims(mel_spec_input, axis=0) 
    
    return mel_spec_input

In [89]:
def predict_audio_genre(audio_file):
    if not audio_file:  
        return "No audio file uploaded."

    spectrogram = preprocess_audio(audio_file)
    if spectrogram is not None:
        prediction = cnn_model.predict(spectrogram)
        predicted_label = np.argmax(prediction, axis=1)
        return f"{label_encoder.inverse_transform(predicted_label)[0]}"
    else:
        return "Error processing audio file"

In [90]:
def predict_audio_genre_fe(audio_file):
    if not audio_file:  
        return "No audio file uploaded."

    spectrogram = preprocess_feature_engineered_spectrogram(audio_file)
    if spectrogram is not None:
        prediction = cnn_featured_model.predict(spectrogram)
        predicted_label = np.argmax(prediction, axis=1)
        return f"{label_encoder.inverse_transform(predicted_label)[0]}"
    else:
        return "Error processing audio file"

In [91]:
def predict_with_ml(audio_file):
    if not audio_file: 
        return "No audio file uploaded."

    features = extract_audio_features(audio_file)
    if features is None:
        return "Error extracting features from audio."

    features_df = pd.DataFrame([features])

    feature_columns = ['Spectral Centroid', 'Spectral Bandwidth', 'Spectral Roll-Off',
                       'Zero Crossing Rate', 'RMS Energy', 'Temporal Entropy']
    features_df = features_df[feature_columns]

    features_scaled = scaler.transform(features_df)

    prediction = rf_model.predict(features_scaled)

    predicted_label = label_encoder.inverse_transform(prediction)

    return f"Predicted Genre: {predicted_label[0]}"

In [92]:
def predict_with_cnn_featured(file_path):
    spectrogram = preprocess_feature_engineered_spectrogram(file_path)
    prediction = cnn_featured_model.predict(spectrogram)
    predicted_label = label_encoder.inverse_transform([np.argmax(prediction)])
    return predicted_label[0]

In [93]:
def predict_genre_gradio(audio_file):
    if not audio_file:
        return "No audio file uploaded."

    return predict_with_ml(audio_file)

In [54]:
interface = gr.Interface(
    fn=predict_audio_genre,
    inputs=gr.Audio(type="filepath", label="Upload Audio File"),
    outputs=gr.Textbox(label="Predicted Genre"),
    title="Music Genre Classification",
    description="Upload an audio file to classify its genre using a CNN model."
)

In [12]:
interface = gr.Interface(
    fn=predict_genre_gradio,
    inputs=gr.Audio(type="filepath", label="Upload Audio File"),
    outputs=gr.Textbox(label="Predicted Genre"),
    title="Music Genre Classification (ML Model)",
    description="Upload an audio file to classify its genre using a Random Forest model trained on audio features."
)

In [96]:
interface = gr.Interface(
    fn=predict_with_cnn_featured,
    inputs=gr.Audio(type="filepath", label="Upload Audio File"),
    outputs=gr.Textbox(label="Predicted Genre"),
    title="Music Genre Classification",
    description="Upload an audio file to classify its genre using Feature Engineered CNN Model"
)

In [97]:
interface.launch()

* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\srira\anaconda3\Lib\site-packages\gradio\queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\srira\anaconda3\Lib\site-packages\gradio\route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\srira\anaconda3\Lib\site-packages\gradio\blocks.py", line 2019, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\srira\anaconda3\Lib\site-packages\gradio\blocks.py", line 1566, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\srira\anaconda3\Lib\site-packages\anyio\to_thread.py", line 28, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(func, *

1/1 [==============================] - 0s 34ms/step


In [19]:
interface.close()

Closing server running on port: 7861
